In [1]:
pip install nemo_toolkit[physics]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 468.2 kB/s eta 0:00:00 kB/s eta 0:00:0101
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 877.1 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 1.1 MB/s eta 0:00:006 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 1.5 MB/s eta 0:00:00MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=b9288faf24712d43cc

In [1]:
# SINGLE-CELL PhysicsNeMo Demo

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

try:
    from nemo.physics.models import PDEModel
    from nemo.physics.domain import Domain
    from nemo.physics.constraints import DataConstraint, PDEConstraint
    from nemo.physics.solvers import Solver
except ImportError as e:
    print("Make sure you have installed PhysicsNeMo, e.g. `pip install nemo_toolkit[physics]`")
    raise e



# LSTM + infiltration

class LSTMInfiltrationReLU(nn.Module):
   
    def __init__(self, input_size, rain_idx, dem_idx, slope_idx,
                 hidden_size=128, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, 1)  # outputs log(Q)

        # PDE infiltration parameters
        self.alpha   = nn.Parameter(torch.tensor(0.01, dtype=torch.float32))
        self.i0      = nn.Parameter(torch.tensor(0.1,  dtype=torch.float32))
        self.iDEM    = nn.Parameter(torch.tensor(0.0,  dtype=torch.float32))
        self.iSlope  = nn.Parameter(torch.tensor(0.0,  dtype=torch.float32))
        self.relu = nn.ReLU()

        self.rain_idx  = rain_idx
        self.dem_idx   = dem_idx
        self.slope_idx = slope_idx

    def forward(self, x):
   
        lstm_out, (h_n, c_n) = self.lstm(x)
        last_out = lstm_out[:, -1, :]  # last time-step
        log_q_pred = self.fc(last_out)
        return log_q_pred

    def infiltration_fn(self, dem, slope):
     
        raw_infil = self.i0 + self.iDEM * dem + self.iSlope * slope
        return self.relu(raw_infil)



# Wrap in a PDEModel

class MyPhysicsModel(PDEModel):
    """
    A PhysicsNeMo-compatible model that wraps the PyTorch LSTMInfiltrationReLU.
    """
    def __init__(self, lstm_model):
        super().__init__()
        self.lstm_model = lstm_model

    def forward(self, inputs):
        x_seq = inputs["x"]  # shape: (batch, seq_len, input_size)
        log_q_pred = self.lstm_model(x_seq)
        return {"logQ": log_q_pred}


def infiltration_pde_residual(model: MyPhysicsModel, inputs, outputs):
    """
    PDE (discrete infiltration):
      Q_{t+1} - Q_t - alpha * (rain_t - infiltration) = 0
    We'll unroll the LSTM to get Q_t for each step.
    """
    x_seq = inputs["x"]
    lstm_model = model.lstm_model

    batch_size, seq_len, _ = x_seq.shape
    hidden = None
    Q_preds = []

    # Unroll the LSTM to get Q_t at each time step
    for t_idx in range(seq_len):
        x_t = x_seq[:, t_idx, :].unsqueeze(1)  # shape: (batch,1,input_size)
        if hidden is None:
            lstm_out, hidden = lstm_model.lstm(x_t)
        else:
            lstm_out, hidden = lstm_model.lstm(x_t, hidden)
        log_q_t = lstm_model.fc(lstm_out[:, -1, :])
        Q_t = torch.expm1(log_q_t)
        Q_preds.append(Q_t)

    alpha = lstm_model.alpha
    residual_list = []
    for t_idx in range(seq_len - 1):
        Q_t  = Q_preds[t_idx]
        Q_t1 = Q_preds[t_idx + 1]

        rain_t  = x_seq[:, t_idx, lstm_model.rain_idx].unsqueeze(1)
        dem_t   = x_seq[:, t_idx, lstm_model.dem_idx].unsqueeze(1)
        slope_t = x_seq[:, t_idx, lstm_model.slope_idx].unsqueeze(1)

        infil_val = lstm_model.infiltration_fn(dem_t, slope_t)
        eff_rain  = torch.clamp(rain_t - infil_val, min=0.0)

        # PDE residual
        # (Q_{t+1} - Q_t - alpha * eff_rain)
        residual_t = Q_t1 - Q_t - alpha * eff_rain
        residual_list.append(residual_t)

    # Combine across timesteps => shape (batch, seq_len-1, 1)
    residual_stack = torch.stack(residual_list, dim=1)
    # We'll return the mean residual so PDEConstraint can drive it to 0
    residual = torch.mean(residual_stack)
    return residual


def get_pde_residual_fn(pde_name: str):
    """
    Return the PDE residual function based on 'pde_name'.
    Modify or add more PDE definitions as you like.
    """
    if pde_name == "infiltration_basic":
        return infiltration_pde_residual
    else:
        raise ValueError(f"Unknown PDE: {pde_name}")


########################################
# 4) Custom Dataset + Collate
########################################
class MyDischargeDataset(Dataset):
    """
    Wrap your existing (X, y, tau, lat, lon) arrays in a dict-based format
    that PhysicsNeMo constraints can consume.
    """
    def __init__(self, X, y_log, tau, lat, lon):
        self.X = X
        self.y_log = y_log  # log(Q)
        self.tau = tau
        self.lat = lat
        self.lon = lon

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            "x": self.X[idx],       # shape: (seq_len, input_size)
            "y_log": self.y_log[idx],  # shape: (1,)
            "tau": self.tau[idx],
            "lat": self.lat[idx],
            "lon": self.lon[idx],
        }


def data_loss_fn(pred, ground_truth):
    """
    Weighted MSE in log-space:
    pred["logQ"] vs ground_truth["y_log"].
    """
    log_q_pred = pred["logQ"]
    log_q_true = ground_truth["y_log"]
    Q_true = torch.expm1(log_q_true)

    beta = 15.0
    global_qmax = 60.0
    w = 1.0 + beta * (Q_true / global_qmax)
    diff = (log_q_pred - log_q_true) ** 2

    return torch.mean(w * diff)


def collate_fn(batch_list):
    """
    Merge a list of sample dicts into one batch dict for PhysicsNeMo constraints.
    """
    x_stacked   = torch.stack([b["x"] for b in batch_list], dim=0)
    y_log_stk   = torch.stack([b["y_log"] for b in batch_list], dim=0)
    tau_stacked = torch.stack([b["tau"] for b in batch_list], dim=0)
    lat_stacked = torch.stack([b["lat"] for b in batch_list], dim=0)
    lon_stacked = torch.stack([b["lon"] for b in batch_list], dim=0)
    
    return {
        "x": x_stacked,     # (batch, seq_len, input_size)
        "y_log": y_log_stk, # (batch,1)
        "tau": tau_stacked,
        "lat": lat_stacked,
        "lon": lon_stacked,
    }


########################################
# 5) Build PhysicsNeMo Domain
########################################
def build_physicsnemo_domain(model, dataset, pde_name="infiltration_basic",
                             batch_size=32, lambda_phys=0.8):
    domain = Domain()

    # -- DataConstraint (supervised) --
    data_constraint = DataConstraint(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        input_keys=("x",),    # we'll pass 'x' to the model
        output_keys=("logQ",),
        loss_fn=data_loss_fn
    )
    domain.add_constraint(data_constraint, name="data_constraint")

    # -- PDEConstraint (physics) --
    pde_fn = get_pde_residual_fn(pde_name)

    # Wrap PDE residual in a function that returns a dict: {residual_key: residual_value}
    def pde_residual_wrap(inputs, outputs):
        return {"pde_res": pde_fn(model, inputs, outputs)}

    pde_constraint = PDEConstraint(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        input_keys=("x",),
        output_keys=("logQ",),
        residual_fn=pde_residual_wrap,
        residual_key="pde_res",
        weight=lambda_phys
    )
    domain.add_constraint(pde_constraint, name="pde_constraint")

    return domain


########################################
# 6) Main Training Function (Solver)
########################################
def train_with_physicsnemo(
    train_ds,
    feature_cols,
    pde_name="infiltration_basic",
    epochs=100,
    lr=1e-4,
    hidden_size=128,
    num_layers=2,
    dropout=0.2,
    batch_size=32,
    lambda_phys=0.8,
    device=None
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Indices for PDE infiltration
    rain_idx  = feature_cols.index("rainrate")
    dem_idx   = feature_cols.index("DEM")
    slope_idx = feature_cols.index("SLOPE")

    # LSTM Model
    lstm_model = LSTMInfiltrationReLU(
        input_size=len(feature_cols),
        rain_idx=rain_idx,
        dem_idx=dem_idx,
        slope_idx=slope_idx,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    # Wrap in PhysicsNeMo PDEModel
    physics_model = MyPhysicsModel(lstm_model).to(device)

    # Build domain (Data + PDE constraints)
    domain = build_physicsnemo_domain(
        physics_model,
        dataset=train_ds,
        pde_name=pde_name,
        batch_size=batch_size,
        lambda_phys=lambda_phys
    )

    # Create solver
    solver = Solver(
        model=physics_model,
        domain=domain,
        optimizer=torch.optim.Adam(physics_model.parameters(), lr=lr),
        max_epochs=epochs,
        device=device,
        # Optional: define how often to log, which loggers to use, etc.
    )

    # Train
    solver.solve()
    return physics_model



def evaluate_physicsnemo_model(physics_model, test_loader):
    physics_model.eval()
    all_preds, all_truth = [], []

    with torch.no_grad():
        for batch_dict in test_loader:
            for k in batch_dict:
                # Move tensors to model device
                if torch.is_tensor(batch_dict[k]):
                    batch_dict[k] = batch_dict[k].to(next(physics_model.parameters()).device)

            # Forward => get pred dict
            pred_dict = physics_model(batch_dict)
            log_q_pred = pred_dict["logQ"]
            q_pred = torch.expm1(log_q_pred).cpu().numpy().flatten()
            q_true = torch.expm1(batch_dict["y_log"]).cpu().numpy().flatten()

            all_preds.append(q_pred)
            all_truth.append(q_true)

    import numpy as np
    all_preds = np.concatenate(all_preds)
    all_truth = np.concatenate(all_truth)

    mse = np.mean((all_preds - all_truth)**2)
    print(f"Test MSE = {mse:.4f}")
    # You can also plot or group by station, etc. as you do in your code.



if __name__ == "__main__":
    # Example usage (pseudo-code):
    # Suppose you have loaded and preprocessed X_train, y_train, etc.
    #
    # feature_cols = ["rainrate", "DEM", "SLOPE", ...] # adapt your columns
    
    # # Create train dataset
    # train_ds = MyDischargeDataset(X_train, y_train_log, tau_train, lat_train, lon_train)
    #
    # # Train
    # physics_model = train_with_physicsnemo(
    #     train_ds=train_ds,
    #     feature_cols=feature_cols,
    #     pde_name="infiltration_basic",
    #     epochs=120,
    #     lr=1e-4,
    #     hidden_size=128,
    #     num_layers=2,
    #     dropout=0.2,
    #     batch_size=32,
    #     lambda_phys=0.8,
    #     device="cuda"
    # )
    #
    # # Evaluate on test data
    # test_ds = MyDischargeDataset(X_test, y_test_log, tau_test, lat_test, lon_test)
    # test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)
    # evaluate_physicsnemo_model(physics_model, test_loader)

    print("Single-cell script loaded. Adapt the code under `if __name__ == \"__main__\":` to run training/evaluation.")


Make sure you have installed PhysicsNeMo, e.g. `pip install nemo_toolkit[physics]`


ModuleNotFoundError: No module named 'nemo.physics'

In [4]:
pip install nvidia-physicsnemo[all]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 912.0 kB/s eta 0:00:001m706.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 943.5 kB/s eta 0:00:001m918.7 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.0 MB/s eta 0:00:001.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 888.7 kB/s eta 0:00:001m854.9 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 813.7 kB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB

In [2]:
pip show nvidia-physicsnemo


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show nemo_toolkit


Name: nemo-toolkit
Version: 2.2.1
Summary: NeMo - a toolkit for Conversational AI
Home-page: https://github.com/nvidia/nemo
Author: NVIDIA
Author-email: NVIDIA <nemo-toolkit@nvidia.com>
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management 

In [4]:
import nemo


In [5]:
import nemo.collections


In [6]:
import nemo.collections.physics  # or maybe nemo.physics


ModuleNotFoundError: No module named 'nemo.collections.physics'

In [7]:
pip show nvidia-physicsnemo


Note: you may need to restart the kernel to use updated packages.


SyntaxError: invalid syntax (2478640907.py, line 1)

In [10]:
import numpy as np 
from physicsnemo.sym.geometry.primitives_3d import Box

ModuleNotFoundError: No module named 'physicsnemo'

In [12]:
pip install "Cython"


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 4.8 MB/s eta 0:00:004.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install nvidia-physicsnemo.sym --no-build-isolation


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.8/521.8 kB 2.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.7/369.7 kB 2.8 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.1/772.1 kB 5.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 6.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.7 MB/s e

In [14]:
pip install nvidia-physicsnemo.sym

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.8/521.8 kB 2.1 MB/s eta 0:00:001.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "/home/amit/anaconda3/envs/gpu/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/home/amit/anaconda3/envs/gpu/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/home/amit/anaconda3/envs/gpu/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks